In [1]:
import uuid
# project_id = str(uuid.uuid4())
run_id = str(uuid.uuid4())
print(run_id)

6d0370b7-7aab-4394-884a-266bc284d07a


In [2]:
# payload-1 (classification Airline Delay dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBClassifier": null}}, 
"dataset": "AIRLINE_DEP_DELAY_10K", 
"target_column": "DEP_DEL15"}}'''.format(run_id)

In [3]:
# payload-2 (Regression Alcohol dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}}, 
"dataset": "ALCOHOL_DATA_10L", 
"target_column": "QUALITY"}}'''.format(run_id)

In [ ]:
# payload-3 (classification Employee dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.linear_model.LogisticRegression": null}}, 
"dataset": "EMPLOYEE_10L", 
"target_column": "LEAVEORNOT"}}'''.format(run_id)

In [ ]:
# payload-4 (Regression Diamonds dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBRegressor": null}}, 
"dataset": "DIAMONDS", 
"target_column": "PRICE"}}'''.format(run_id)

In [3]:
exp_data

'{\n"source":"EXP", \n"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", \n"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", \n"run_id":"6d0370b7-7aab-4394-884a-266bc284d07a", \n"exp_name": "Final_recipe", \n"algorithm_type":"classification", \n"algo_details": {"snowflake.ml.modeling.xgboost.XGBClassifier": null}, \n"dataset": "AIRLINE_DEP_DELAY_10K", \n"target_column": "DEP_DEL15"}'

In [39]:
from snowflake.snowpark.session import Session
from snow_exp import run_experiment
import snow_exp as sf
import json, asyncio

In [44]:
#API function
def execute_sf_sproc(payload, sproc_name="run_experiment"):
#     try:
#         task = asyncio.create_task(initiate_sproc_process(payload))
    task = initiate_sproc_process(payload)
    print(task)
    print("Success")
#     except Exception as ex:    
#         return "Failed: ",ex

def initiate_sproc_process(payload, sproc_name="run_experiment"):
    exp_details = json.loads(payload)
    print("Creating Snowflake Session object...")
    session = sf.get_session(exp_details.get("dataset"),exp_details.get("project_id"))
    print("Session has been created !")
    stage = sf.create_stage(session)
    print("Creating stored procedure...")
    create_sproc(session, stage, run_experiment)
    print("Stored procedure has been created successfully!")
    print("Executing Procedure")
#     session.add_import("/notebooks/notebooks/SnowparkRnD/snow_exp.py", import_path="notebooks.notebooks.SnowparkRnD.snow_exp")
    sproc_response = session.call(sproc_name, payload)
#     sproc_response = sf.run_experiment(session,payload)
    print("Stored Procedure Executed Successfully !")
    return sproc_response

def create_sproc(session, stage, fnc, func_name="run_experiment"):
    print("Creating stored procedure...")
    session.custom_package_usage_config['enabled'] = True
    session.sproc.register(func=fnc,
                           name=func_name,
                           packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn"],
                           isPermanant=False,
                           stage_location=stage,
                           replace=True)
    print("Stored procedure has been created successfully!")

In [45]:
execute_sf_sproc(exp_data)

Creating Snowflake Session object...
Session has been created !
Creating stored procedure...
Creating stored procedure...


Stored procedure has been created successfully!
Stored procedure has been created successfully!
Executing Procedure


SnowparkSQLException: (1304): 01b56ba1-0000-6f57-0000-576d072cd08a: 100357 (P0000): Python Interpreter Error:
ModuleNotFoundError: No module named 'snow_exp' in function RUN_EXPERIMENT with handler udf_py_1474115455.compute

In [7]:
print(a)

('Failed: ', RuntimeError('asyncio.run() cannot be called from a running event loop'))


In [1]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
exp_details=json.loads(exp_data)
session = get_session(exp_details.get("dataset"),exp_details.get("project_id"))
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.custom_package_usage_config['enabled'] = True
session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# tagging session
print("Setting tag to session object: tag= ", run_id)
session.query_tag=run_id

print("Executing Procedure")
procedure_response = session.call("run_experiment", exp_data)
# procedure_response = run_experiment(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")

Creating Snowflake Session object...


NameError: name 'json' is not defined